In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib 
plt.style.use("ggplot")

In [ ]:
# Preprocesado y modelado
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.model_selection import train_test_split
# Configuración warnings
import warnings
warnings.filterwarnings('once')

In [ ]:
# Algoritmos de ensemble
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm 
import catboost
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [ ]:
## Optimización de hiperparámetros
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
import multiprocessing
from multiprocessing import Pool
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold

In [ ]:
## Métricas de evaluación
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer

In [ ]:
df= pd.read_csv('/kaggle/input/ictus-data-set/stroke_dataset.csv')
df.head(5)

In [ ]:
for i in  range(len(df)):
    if df["gender"][i]=='Male':
        df["gender"][i]='0'
    else :
        df["gender"][i]='1'

df[[ "gender" ]] = df[[ "gender" ]].astype(int)
df.gender.value_counts()

In [ ]:
for i in  range(len(df)):
    if df["Residence_type"][i]=='Rural':
        df["Residence_type"][i]='0'
    else :
        df["Residence_type"][i]='1'
        
df[[ "Residence_type" ]] = df[[ "Residence_type" ]].astype(int)
df.Residence_type.value_counts()

In [ ]:
for i in  range(len(df)):
    if df["ever_married"][i]=='No':
        df["ever_married"][i]='0'
    else :
        df["ever_married"][i]='1'
        
df[[ "ever_married" ]] = df[[ "ever_married" ]].astype(int)
df.ever_married.value_counts()

In [ ]:
df[[ "stroke" ]] = df[[ "stroke" ]].astype(int)
df.stroke.value_counts()

In [ ]:
df[[ "hypertension", "heart_disease" ]] = df[[ "hypertension", "heart_disease" ]].astype('bool').astype(int)
df.info()

In [ ]:
df['age']= df['age'].round()
df[["age"]]= df[["age"]].astype(int)
df.info()

In [ ]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder , LabelEncoder
from sklearn.compose import ColumnTransformer

numericas = ["age", "avg_glucose_level" , "bmi" , "hypertension", "heart_disease"]
categoricas= ["work_type" , "smoking_status" , "gender", "ever_married" , "Residence_type"]

#Configuro los transformer
transformer_numerico = ("transformer_numerico",MinMaxScaler(), numericas) 
transformer_categorico = ("transformer_categorico",OneHotEncoder(), categoricas)

#Defino el transformer
transformer = ColumnTransformer ([transformer_numerico, transformer_categorico ], remainder="passthrough")

In [ ]:
X = df.drop('stroke',axis=1)
y= df['stroke']
X_ub = transformer.fit_transform(X) 

In [ ]:
from imblearn.under_sampling import RandomUnderSampler  
from sklearn.model_selection import train_test_split

rus = RandomUnderSampler(sampling_strategy=1) # Float 
X_rus, y_rus = rus.fit_resample(X,y)  
X_rus = transformer.fit_transform(X_rus) 
X_train_rus, X_test_rus, y_train_rus, y_test_rus = train_test_split(X_rus, y_rus, test_size=0.20, random_state=101, stratify=y_rus)

# Algoritmos ensemble con balanceo de datos ---->> RandomUnderSampler

# RamdomForestClassifier -----> Grid Search basado en validación cruzada

In [ ]:
# Grid de hiperparámetros evaluados RandomForestClassifier
# ==============================================================================
param_grid = {'n_estimators'  : [50, 100, 500, 1000],
              'criterion'     : ['gini', 'entropy', 'log_loss'],
              'max_features'  : ['auto', 'sqrt', 'log2'],
              #'max_depth'     : [None, 1, 3, 5, 10, 20]
             }

# Búsqueda por grid search con validación cruzada RandomForetsClassifier
# ==============================================================================
grid_RFC = GridSearchCV(
        estimator  = RandomForestClassifier(random_state=123),
        param_grid = param_grid,
        scoring    = 'accuracy',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

grid_RFC.fit(X = X_train_bal, y = y_train_bal)

# Resultados RandomForetsClassifier
# ==============================================================================
print("Resultados GridSearch ---->>> RandomForestClassifier")
print()
resultados = pd.DataFrame(grid_RFC.cv_results_)
resultados.filter(regex = '(param*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

# Mejores hiperparámetros por validación cruzada RandomForestClassifier
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv) RandomForestClassifier")
print("----------------------------------------")
print(grid_RFC.best_params_, ":", grid_RFC.best_score_, grid_RFC.scoring)

# RFC con optimización de hiperparámetros
modelo_RFC_bal = grid_RFC.best_estimator_
#Accuracy del modelo
RFC_accuracy = grid_RFC.best_score_

In [ ]:
# Cross Validation
cv_RFC_bal = cross_validate(modelo_RFC_bal, X_bal, y_bal, cv=10)
sorted(cv_RFC_bal.keys())
cv_RFC_bal['test_score']

In [ ]:
# Accuracy cross_validatation
cv_RFC_mean_bal =sum(cv_RFC_bal['test_score'])/len(cv_RFC_bal['test_score'])

In [ ]:
#Prediccion y evaluacion del modelo
y_pred_bal= modelo_RFC_bal.predict(X_test_bal)
y_pred_train_bal= modelo_RFC_bal.predict(X_train_bal)

#Accuracy del modelo
accuracy_test_predict= accuracy_score(y_test_bal, y_pred_bal)
#Accuracy del train
accuracy_train_bal = accuracy_score(y_train_bal,y_pred_train_bal)

print("")
print("El accuracy de test es: {:.2f} %".format(100 * accuracy_test_predict))
print("")
print("El accuracy del train es: {:.2f} %".format(100 * accuracy_train_bal))
print("")
print("El overfitting del test y train es {:.2f} % ".format(((accuracy_test_predict - accuracy_train_bal)/accuracy_test_predict) *100))
print("")
print("El accuracy del cross_validate es: {:.2f} %".format(100 * cv_RFC_mean_bal))
print("")
print("El overfitting del cross_validate (test) y train es {:.2f} % ".format(((cv_RFC_mean_bal - accuracy_train_bal)/accuracy_test_predict) *100))
print("")
print("")

#definimos funciona para mostrar los resultados
print("Matriz de confusión")
print("")

Labels= 'No_Stroke', 'Stroke'

def mostrar_resultados(y_test_bal, y_pred_bal):
    conf_matrix = confusion_matrix(y_test_bal, y_pred_bal)
    plt.figure(figsize=(8, 8))
    sns.heatmap(conf_matrix, xticklabels=Labels, yticklabels=Labels, annot=True, fmt="d");
    plt.title("Matriz de confusión")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.show()
    print("")
    print("Reporte de Clasificación (test)")
    print("-------------------")
    print (classification_report(y_test_bal, y_pred_bal))
    
mostrar_resultados(y_test_bal, y_pred_bal)
print("")
print("Reporte de Clasificación (train)")
print("-------------------")
print (classification_report(y_train_bal,y_pred_train_bal))
print("")

In [ ]:
#Accuracy de las predicciones sobre test
accuracy_test_predict= accuracy_score(y_test_bal, y_pred_bal)
#Accuracy de las predicciones sobre train (para overfitting)
accuracy_train_bal= accuracy_score(y_train_bal, y_pred_train_bal)

# AdaBoostClassifier -----> Grid Search basado en validación cruzada

In [ ]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'n_estimators'  : [50, 100, 500, 1000],
              'algorithm'     : ['SAMME', 'SAMME.R'],
              'learning_rate' : [0.001, 0.01, 0.1]
             }

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid_ADBC = GridSearchCV(
        estimator  = AdaBoostClassifier (random_state=123),
        param_grid = param_grid,
        scoring    = 'accuracy',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

#ahora hacemos el entrenamiento del modelo
grid_ADBC.fit(X = X_train_bal, y = y_train_bal)

# Resultados
# ==============================================================================
print("Resultados GridSearch ---->>> AdaBoostClassifier")
resultados = pd.DataFrame(grid_ADBC.cv_results_)
resultados.filter(regex = '(param*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

# Mejores hiperparámetros por validación cruzada AdaBoostClassifier
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv) AdaBoostClassifier")
print("----------------------------------------")
print(grid_ADBC.best_params_, ":", grid_ADBC.best_score_, grid_ADBC.scoring)

#Guardamos el mejor modelo y su accuracy
modelo_ADBC_bal = grid_ADBC.best_estimator_
#Accuracy del modelo
RFC_accuracy = grid_RFC.best_score_


In [ ]:
# Cross validation
cv_ADBC_bal = cross_validate(modelo_ADBC_bal, X_bal, y_bal, cv=10)
sorted(cv_ADBC_bal.keys())
cv_ADBC_bal['test_score']

In [ ]:
# Accuracy cross_validation
cv_ADBC_mean_bal =sum(cv_ADBC_bal['test_score'])/len(cv_ADBC_bal['test_score'])

In [ ]:
# Prediccion y evaluacion del modelo
y_pred_bal= modelo_ADBC_bal.predict(X_test_bal)
y_pred_train_bal= modelo_ADBC_bal.predict(X_train_bal)

#Accuracy del modelo
accuracy_test_predict= accuracy_score(y_test_bal, y_pred_bal)
#Accuracy del train
accuracy_train_bal = accuracy_score(y_train_bal,y_pred_train_bal)

print("")
print("El accuracy de test es: {:.2f} %".format(100 * accuracy_test_predict))
print("")
print("El accuracy del train es: {:.2f} %".format(100 * accuracy_train_bal))
print("")
print("El overfitting del test y train es {:.2f} % ".format(((accuracy_test_predict - accuracy_train_bal)/accuracy_test_predict) *100))
print("")
print("El accuracy del cross_validate es: {:.2f} %".format(100 * cv_ADBC_mean_bal))
print("")
print("El overfitting del cross_validate (test) y train es {:.2f} % ".format(((cv_ADBC_mean_bal - accuracy_train_bal)/accuracy_test_predict) *100))
print("")
print("")

#definimos funciona para mostrar los resultados
print("Matriz de confusión")
print("")

Labels= 'No_Stroke', 'Stroke'

def mostrar_resultados(y_test_bal, y_pred_bal):
    conf_matrix = confusion_matrix(y_test_bal, y_pred_bal)
    plt.figure(figsize=(8, 8))
    sns.heatmap(conf_matrix, xticklabels=Labels, yticklabels=Labels, annot=True, fmt="d");
    plt.title("Matriz de confusión")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.show()
    print("")
    print("Reporte de Clasificación (test)")
    print("-------------------")
    print (classification_report(y_test_bal, y_pred_bal))
    
mostrar_resultados(y_test_bal, y_pred_bal)
print("")
print("Reporte de Clasificación (train)")
print("-------------------")
print (classification_report(y_train_bal,y_pred_train_bal))
print("")

# XGBoostClassifier -----> Grid Search basado en validación cruzada

In [ ]:
# Grid de hiperparámetros evaluados XGBoostClassifier
# ==============================================================================
param_grid = {'n_estimators'  : [50, 100, 500, 1000],
              'max_depth'     : [None, 1, 3, 5, 10, 20],
              'subsample'     : [0.5, 1],
              'learning_rate' : [0.001, 0.01, 0.1]
             }

# Búsqueda por grid search con validación cruzada XGBoostClassifier
# ==============================================================================
grid_XGBC = GridSearchCV(
        estimator  = XGBClassifier(random_state=123),
        param_grid = param_grid,
        scoring    = 'accuracy',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

grid_XGBC.fit(X = X_train_bal, y = y_train_bal)

# Resultados XGBoostClassifier
# ==============================================================================
print("Resultados GridSearch ---->>> XGBClassifier")
print()
resultados = pd.DataFrame(grid_XGBC.cv_results_)
resultados.filter(regex = '(param*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

# Mejores hiperparámetros por validación cruzada XGBClassifier
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv) XGBClassifier")
print("----------------------------------------")
print(grid_XGBC.best_params_, ":", grid_XGBC.best_score_, grid_XGBC.scoring)

# Guardamos el mejor modelo y el accuracy
modelo_XGBC_bal = grid_XGBC.best_estimator_
#Accuracy del modelo
RFC_accuracy = grid_RFC.best_score_

In [ ]:
# Cross Validation
cv_XGBC_bal = cross_validate(modelo_XGBC_bal, X_bal, y_bal, cv=10)
sorted(cv_XGBC_bal.keys())
cv_XGBC_bal['test_score']

In [ ]:
# Accuracy cross_validation
cv_XGBC_mean_bal =sum(cv_XGBC_bal['test_score'])/len(cv_XGBC_bal['test_score'])

In [ ]:
# Predicciones y evaluación del modelo (overfitting)
y_pred_bal= modelo_XGBC_bal.predict(X_test_bal)
y_pred_train_bal= modelo_XGBC_bal.predict(X_train_bal)

#Accuracy del modelo
accuracy_test_predict= accuracy_score(y_test_bal, y_pred_bal)
#Accuracy del train
accuracy_train_bal = accuracy_score(y_train_bal,y_pred_train_bal)

print("")
print("El accuracy de test es: {:.2f} %".format(100 * accuracy_test_predict))
print("")
print("El accuracy del train es: {:.2f} %".format(100 * accuracy_train_bal))
print("")
print("El overfitting del test y train es {:.2f} % ".format(((accuracy_test_predict - accuracy_train_bal)/accuracy_test_predict) *100))
print("")
print("El accuracy del cross_validate es: {:.2f} %".format(100 * cv_XGBC_mean_bal))
print("")
print("El overfitting del cross_validate (test) y train es {:.2f} % ".format(((cv_XGBC_mean_bal - accuracy_train_bal)/accuracy_test_predict) *100))
print("")
print("")

#definimos funciona para mostrar los resultados
print("Matriz de confusión")
print("")

Labels= 'No_Stroke', 'Stroke'

def mostrar_resultados(y_test_bal, y_pred_bal):
    conf_matrix = confusion_matrix(y_test_bal, y_pred_bal)
    plt.figure(figsize=(8, 8))
    sns.heatmap(conf_matrix, xticklabels=Labels, yticklabels=Labels, annot=True, fmt="d");
    plt.title("Matriz de confusión")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.show()
    print("")
    print("Reporte de Clasificación (test)")
    print("-------------------")
    print (classification_report(y_test_bal, y_pred_bal))
    
mostrar_resultados(y_test_bal, y_pred_bal)
print("")
print("Reporte de Clasificación (train)")
print("-------------------")
print (classification_report(y_train_bal,y_pred_train_bal))
print("")

# LGBMClassifier

In [ ]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {
    'learning_rate': [0.005, 0.01],
    'n_estimators': [8,16,24],
    'num_leaves': [6,8,12,16], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['gbdt', 'dart'], # for better accuracy -> try dart
    'objective' : ['binary'],
    'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
    'colsample_bytree' : [0.64, 0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid_LGBMC = GridSearchCV(
        estimator  = LGBMClassifier (random_state=123),
        param_grid = param_grid,
        scoring    = 'accuracy',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

#ahora hacemos el entrenamiento del modelo
grid_LGBMC.fit(X = X_train_bal, y = y_train_bal)

# Resultados
# ==============================================================================
print("Resultados GridSearch ---->>> LGBMClassifier")
resultados = pd.DataFrame(grid_LGBMC.cv_results_)
resultados.filter(regex = '(param*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

# Mejores hiperparámetros por validación cruzada LGBMClassifier
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv) LGBMClassifier")
print("----------------------------------------")
print(grid_LGBMC.best_params_, ":", grid_LGBMC.best_score_, grid_LGBMC.scoring)

# Guardamos el mejor modelo y el accuracy
modelo_LGBMC_bal = grid_LGBMC.best_estimator_
# Accuracy del modelo
LGBMC_accuracy = grid_LGBMC.best_score_

In [ ]:
# Cross_validation
cv_LGBMC_bal = cross_validate(modelo_LGBMC_bal, X_bal, y_bal, cv=10)
sorted(cv_LGBMC_bal.keys())
cv_LGBMC_bal['test_score']

In [ ]:
# Accuracy cross_validation
cv_LGBMC_mean_bal =sum(cv_LGBMC_bal['test_score'])/len(cv_LGBMC_bal['test_score'])

In [ ]:
# Prediccion y evaluacion del modelo
y_pred_bal= modelo_LGBMC_bal.predict(X_test_bal)
y_pred_train_bal= modelo_LGBMC_bal.predict(X_train_bal)

#Accuracy del modelo
accuracy_test_predict= accuracy_score(y_test_bal, y_pred_bal)
#Accuracy del train
accuracy_train_bal = accuracy_score(y_train_bal,y_pred_train_bal)

print("")
print("El accuracy de test es: {:.2f} %".format(100 * accuracy_test_predict))
print("")
print("El accuracy del train es: {:.2f} %".format(100 * accuracy_train_bal))
print("")
print("El overfitting del test y train es {:.2f} % ".format(((accuracy_test_predict - accuracy_train_bal)/accuracy_test_predict) *100))
print("")
print("El accuracy del cross_validate es: {:.2f} %".format(100 * cv_LGBMC_mean_bal))
print("")
print("El overfitting del cross_validate (test) y train es {:.2f} % ".format(((cv_LGBMC_mean_bal - accuracy_train_bal)/accuracy_test_predict) *100))
print("")
print("")

#definimos funciona para mostrar los resultados
print("Matriz de confusión")
print("")

Labels= 'No_Stroke', 'Stroke'

def mostrar_resultados(y_test_bal, y_pred_bal):
    conf_matrix = confusion_matrix(y_test_bal, y_pred_bal)
    plt.figure(figsize=(8, 8))
    sns.heatmap(conf_matrix, xticklabels=Labels, yticklabels=Labels, annot=True, fmt="d");
    plt.title("Matriz de confusión")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.show()
    print("")
    print("Reporte de Clasificación (test)")
    print("-------------------")
    print (classification_report(y_test_bal, y_pred_bal))
    
mostrar_resultados(y_test_bal, y_pred_bal)
print("")
print("Reporte de Clasificación (train)")
print("-------------------")
print (classification_report(y_train_bal,y_pred_train_bal))
print("")

# CatBoostClassifier -----> Grid Search basado en validación cruzada

In [ ]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'depth'         : [4,5,6,7,8,9, 10],
              'learning_rate' : [0.01,0.02,0.03,0.04],
              'iterations'    : [10, 20,30,40,50,60,70,80,90, 100]
             }

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid_CBC = GridSearchCV(
        estimator  = CatBoostClassifier(random_state=123),
        param_grid = param_grid,
        scoring    = 'accuracy',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

grid_CBC.fit(X = X_train_bal, y = y_train_bal)

# Resultados
# ==============================================================================
print("Resultados GridSearch ---->>> CatBoostClassifier")
resultados = pd.DataFrame(grid_CBC.cv_results_)
resultados.filter(regex = '(param*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

# Mejores hiperparámetros por validación cruzada CatBoostClassifier
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv) CatBoostClassifier")
print("----------------------------------------")
print(grid_CBC.best_params_, ":", grid_CBC.best_score_, grid_CBC.scoring)

# Mejores hiperparámetros por validación cruzada CatBoostClassifier
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv) CatBoostClassifier")
print("----------------------------------------")
print(grid_CBC.best_params_, ":", grid_CBC.best_score_, grid_CBC.scoring)

# Guardar el mejor modelo y el accuracy
modelo_CBC_bal = grid_CBC.best_estimator_
#Accuracy del modelo
RFC_accuracy = grid_RFC.best_score_

In [ ]:
# Cross_validate CatBoostClassifier
cv_CBC_bal = cross_validate(modelo_CBC_bal, X_bal, y_bal, cv=10)
sorted(cv_CBC_bal.keys())
cv_CBC_bal['test_score']

In [ ]:
# Accuracy cross_validate)
cv_CBC_mean_bal =sum(cv_CBC_bal['test_score'])/len(cv_CBC_bal['test_score'])

In [ ]:
# Prediccion y evaluacion del modelo CatBoostClassifier
y_pred_bal= modelo_CBC_bal.predict(X_test_bal)
y_pred_train_bal= modelo_CBC_bal.predict(X_train_bal)

#Accuracy del modelo
accuracy_test_predict= accuracy_score(y_test_bal, y_pred_bal)
#Accuracy del train
accuracy_train_bal = accuracy_score(y_train_bal,y_pred_train_bal)

print("")
print("El accuracy de test es: {:.2f} %".format(100 * accuracy_test_predict))
print("")
print("El accuracy del train es: {:.2f} %".format(100 * accuracy_train_bal))
print("")
print("El overfitting del test y train es {:.2f} % ".format(((accuracy_test_predict - accuracy_train_bal)/accuracy_test_predict) *100))
print("")
print("El accuracy del cross_validate es: {:.2f} %".format(100 * cv_CBC_mean_bal))
print("")
print("El overfitting del cross_validate (test) y train es {:.2f} % ".format(((cv_CBC_mean_bal - accuracy_train_bal)/accuracy_test_predict) *100))
print("")
print("")

#definimos funciona para mostrar los resultados
print("Matriz de confusión")
print("")

Labels= 'No_Stroke', 'Stroke'

def mostrar_resultados(y_test_bal, y_pred_bal):
    conf_matrix = confusion_matrix(y_test_bal, y_pred_bal)
    plt.figure(figsize=(8, 8))
    sns.heatmap(conf_matrix, xticklabels=Labels, yticklabels=Labels, annot=True, fmt="d");
    plt.title("Matriz de confusión")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.show()
    print("")
    print("Reporte de Clasificación (test)")
    print("-------------------")
    print (classification_report(y_test_bal, y_pred_bal))
    
mostrar_resultados(y_test_bal, y_pred_bal)
print("")
print("Reporte de Clasificación (train)")
print("-------------------")
print (classification_report(y_train_bal,y_pred_train_bal))
print("")